### Applying the stride split to all markers (putting it all together)
So far we only worked with the Poll marker to look at data. Now lets apply the data imputation, filter and stride split on all of our markers. Beacuse the filter will not work on non-finite data, we use the gapfill without thinking about the maximum fill length. Once the filtering is done we will put back the nan values. Once the data has been filtered we will apply the stride split to look at data stride by stride. 


In [1]:
# print(peak_indices)

# Loop over ALL markers
all_markers = []
all_markers.extend(markers_upper_body)
all_markers.extend(markers_stride_split)

# Prepare figure
fig = make_subplots(rows=len(all_markers), cols=1, subplot_titles=all_markers, vertical_spacing=0.04)

for mcnt, marker in enumerate(all_markers):
    # Get raw data from data frame
    raw = df[f"{marker}_z"][start_index:end_index]
    
    # Gapfill, but remember indices with nan
    nan_indices = np.isnan(raw)
    filled, _ = InterpolationEnum.pchip.fill_missing(raw, max_gap_length=np.inf)
     
    # Filter use our function, lets use cutoffs here that we know will work for most trotting horses
    filtered = sos_bandpass_filter(data=filled, frame_rate=fs, low_cut=1, high_cut=6, order=4)
    
    # Put back nans
    filtered[nan_indices] = np.nan
    
    #Use the stride split we made
    for pcnt, peak_ind in enumerate(range(len(peak_indices) - 1)):
        stride_start = peak_indices[peak_ind]
        stride_end = peak_indices[peak_ind + 1]
        stride_ts = filtered[stride_start:stride_end]
        
        fig.add_trace(go.Scatter(y=stride_ts,
                                 name=f"{marker} stride {pcnt}",
                                 line=go.scatter.Line(color=colors[mcnt])), row=mcnt + 1, col=1)
        fig.update_yaxes(title="Amplitude (mm)", row=mcnt + 1, col=1)

fig.update_xaxes(title="% of stride", row=len(all_markers), col=1)
fig.update_layout(height=1000)    
fig.show()

NameError: name 'markers_upper_body' is not defined